# Meta Learning Cache Replacement Policy

## Install Dependency

In [1]:
import sys
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm 
from collections import Counter, deque, defaultdict
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
import pickle


## Block Cache Model

### Global Variable

In [2]:
# Maximum block number
maxpos = 1000000000000

# Number of features (Recency, Frequency, Block No.)
num_params = 3

# Cache Size
cache_size = 10

# Sequence Length
sequence_length = 5

# Storage
sequence_data = np.zeros((sequence_length, cache_size, num_params))

# How often to store the data (Removed currently)
sampling_freq = cache_size

# How many % of cache to use
eviction = int(0.7 * cache_size)  

# Results
lruCorrect = 0
lruIncorrect = 0

lfuCorrect = 0
lfuIncorrect = 0

# Variables
X = np.array([], dtype=np.int64).reshape(0,num_params)
Y = np.array([], dtype=np.int64).reshape(0,1)

### Load workload
**cheetah.cs.fiu.edu-110108-113008.1.blkparse** does not contain correct data.

In [3]:
train_file = "FIU_trace/casa-110108-112108.1.blkparse"
read_portion = 0.01


train_data_file = 'test.pkl'

In [4]:
df = pd.read_csv(train_file, sep=' ',header = None)
df.columns = ['timestamp','pid','pname','blockNo', \
              'blockSize', 'readOrWrite', 'bdMajor', 'bdMinor', 'hash']

trainBlockTrace = df['blockNo'].tolist()
trainBlockTrace = trainBlockTrace[:int(len(trainBlockTrace) * read_portion)]
len(trainBlockTrace)

9727

### Data Preprocessing and Data Construction

In [5]:
# Taken from Shehbaz
def get_recency(lru, cache):
    recency = []
    recency_dict = defaultdict(int)
    
    # Compute the recency order of each page in cache
    for time in range(len(lru)):
        recency_dict[lru[time]] = time + 1
        
    for block in cache:
        recency.append(recency_dict[block])

    return recency

def get_frequency(lfu, cache):
    frequency = []
    
    for block in cache:
        frequency.append(lfu[block])
    return frequency

def normalize_columns(input):
    return normalize(input, axis=0)

def identity(input):
    return input

In [6]:
def get_single_length_input(lfu, lru, cache, cache_size, preprocess_func):
    input_recency = get_recency(lru, cache)
    input_frequency = get_frequency(lfu, cache)
    input_block_num = cache[:]
    
    if len(input_recency) < cache_size:
        for i in range(0, cache_size - len(input_recency)):
            input_recency.append(0)
            input_frequency.append(0)
            input_block_num.append(0)
    
    # Columns: recency, frequency, block number
    # Row: cache location
    raw_input = np.column_stack((input_recency, input_frequency, input_block_num))
    
    return preprocess_func(raw_input)


SEQ_DIM = 0
def get_multiple_length_input(sequence_length, prev_inputs, lfu, lru, cache, cache_size, preprocess_func):
    assert prev_inputs.shape[SEQ_DIM] == sequence_length
    current_input = get_single_length_input(lfu, lru, cache, cache_size, preprocess_func)
    return np.vstack((prev_inputs[1:], current_input[None]))
    

def get_outputs(cache, delete):
    assert(len(cache) == len(delete))
    Y_current = []
    KV_sorted = Counter(delete)
    evict_dict = dict(KV_sorted.most_common(eviction))
    
    assert(len(evict_dict) == eviction)
    all_vals = evict_dict.values()
    for e in cache:
        if e in evict_dict.values():
            Y_current.append(1)
        else:
            Y_current.append(0)

    assert(Y_current.count(1) == eviction)
    assert((set(all_vals)).issubset(set(cache)))
    return Y_current


def store_pair(request_time, seq_len, seq_data, lfu, lru, cache, deleted, cache_size, preprocess_func):
    global X,Y,Z
    cache = list(cache)
    Y_current = get_outputs(cache, deleted)
    
    Y = np.vstack((Y, np.array(Y_current)[None]))
    X = np.vstack((X, seq_data[None]))
    Z = np.vstack((Z, request_time))

    assert(Y_current.count(1) == eviction)
    return Y_current

### Belady Optimal Algorithm (From Shehbaz)

In [7]:
def belady_opt(blocktrace, frame):
    global maxpos, sequence_data, sequence_length
    
    optimal = defaultdict(deque)
    deleted = defaultdict(int)
    lfu = defaultdict(int)
    lru = []

    # Build the whole index for finding optimal eviction ordering
    for request_time, block in enumerate(tqdm(blocktrace, desc="OPT: building index")):
        optimal[block].append(request_time)

    hit, miss = 0, 0

    cache = []
    
    for request_time, block in enumerate(tqdm(blocktrace, desc="OPT")):
        old_seq = sequence_data[:]
        # increase frequency count
        lfu[block] +=1

        # Remove the block i at time step j from the index
        if len(optimal[block]) is not 0 and optimal[block][0] == request_time:
            optimal[block].popleft()

        
        if block in cache:
            # Cache Hit
            # Update block to MRU position
            hit += 1
            lru.remove(block)
            lru.append(block)
            
            assert request_time in deleted
            
            del deleted[request_time]
            if len(optimal[block]) is not 0:
                deleted[optimal[block][0]] = block
                optimal[block].popleft()
            else:
                deleted[maxpos] = block
                maxpos -= 1
            sequence_data = get_multiple_length_input(sequence_length, sequence_data, lfu, lru, cache, cache_size, normalize_columns)
        else:
            # Cache Miss
            miss += 1
            if len(cache) == frame:
                # Cache is full
                assert(len(deleted) == frame)
                evictpos = max(deleted)
                
                # Make certain amount of cache as valid eviction candidates
                y_opt = store_pair(request_time, sequence_length, sequence_data, lfu, lru, cache, deleted, cache_size, normalize_columns)
    
                cache[cache.index(deleted[evictpos])] = block
                lru.remove(deleted[evictpos])
                del deleted[evictpos]
            else:
                # Cache isn't full
                cache.append(block)                
            
            # Add the candidate victim page to the j'th time step
            if len(optimal[block]) is not 0:
                deleted[optimal[block][0]] = block
                optimal[block].popleft()
            else:
                deleted[maxpos] = block
                maxpos -= 1
            lru.append(block)
            sequence_data = get_multiple_length_input(sequence_length, sequence_data, lfu, lru, cache, cache_size, normalize_columns)


    hitrate = hit / (hit + miss)

    return hitrate

## Generate training data

In [8]:
X = np.array([], dtype=np.int64).reshape(0, sequence_length, cache_size, num_params)
Y = np.array([], dtype=np.int64).reshape(0, cache_size)
Z = np.array([], dtype=np.int32).reshape(0, 1)
trainHitrate = belady_opt(trainBlockTrace, cache_size)

with open(training_data_file, 'wb+') as f:
    pickle.dump([X, Y, Z], f)

TypeError: get_multiple_length_input() missing 1 required positional argument: 'cache_size'